<a href="https://colab.research.google.com/github/LCaravaggio/scrapers/blob/master/Google_Earth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%%capture
!pip install -U seleniumbase
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!sudo dpkg -i google-chrome-stable_current_amd64.deb
!sudo apt-get install -f  # Para resolver dependencias

In [24]:
import pandas as pd
ciudades=pd.read_csv('ciudades_latlon_2023.csv')

In [44]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from seleniumbase import Driver
import time

def take_image(X):
  # Iniciar el driver
  driver = Driver(uc=True)
  lat=str(ciudades.Latitude[X])
  lon=str(ciudades.Longitude[X])
  # Cargar la página
  url = f"https://earth.google.com/web/@{lat},{lon},20a,200000d,1y,-0h,0t,0r/data=CgRCAggBOgMKATBCAggASg0I____________ARAA"
  driver.get(url)

  start_time = time.time()
  while time.time() - start_time < 60:
      pass

  # Tomar captura de pantalla
  screenshot_name = ciudades.City[X]+".png"
  driver.save_screenshot(screenshot_name)
  print(f"\nScreenshot saved to: {screenshot_name}")

  driver.quit()

In [55]:
take_image(0)


Screenshot saved to: AR: Capital Federal-Ciudad de Buenos Aires.png


In [59]:
import numpy as np
import cv2
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from sklearn.metrics.pairwise import cosine_similarity

# Cargar modelo ResNet50 preentrenado
model = ResNet50(weights="imagenet", include_top=False, pooling="avg")

def extract_features(img_array):
    """Extrae características profundas usando ResNet50."""
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    features = model.predict(img_array)
    return features.flatten()

def compute_inequality_score(img_path, grid_size=4):
    """Divide la imagen en cuadrantes y mide desigualdad en función de las diferencias de características."""

    img = image.load_img(img_path, target_size=(224, 224))  # Redimensionar la imagen
    img_array = image.img_to_array(img)

    # Dividir en cuadrantes
    h, w, _ = img_array.shape
    step_h, step_w = h // grid_size, w // grid_size
    features_list = []

    for i in range(grid_size):
        for j in range(grid_size):
            sub_img = img_array[i*step_h:(i+1)*step_h, j*step_w:(j+1)*step_w, :]
            features = extract_features(sub_img)
            features_list.append(features)

    features_list = np.array(features_list)

    # Calcular diferencias entre cuadrantes
    similarity_matrix = cosine_similarity(features_list)
    avg_similarity = np.mean(similarity_matrix)

    # Convertir a escala de desigualdad (0 = homogéneo, 100 = muy desigual)
    inequality_score = (1 - avg_similarity) * 100

    return round(inequality_score, 2)

# Ejemplo de uso
img_path = ciudades.City[0]+".png"
score = compute_inequality_score(img_path)
print(f"Índice de desigualdad estimado: {score}/100")

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
Índice de desigualdad estimado: 43.68000030517578/100
